In [3]:
import os
import base64
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import WhatsAppChatLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.callbacks import get_openai_callback
from langchain.chains import RetrievalQA
# from langchain.llms import OpenAI

In [4]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.utilities import GoogleSearchAPIWrapper

In [5]:
# OpenAI saved encoded in base64 in a single-line file at data folder
with open("../data/secrets.txt") as secrets:
    open_ai_key = secrets.readline()
    open_ai_key = base64.b64decode(open_ai_key).decode('ascii')
os.environ["OPENAI_API_KEY"] = open_ai_key

In [6]:
loader = WhatsAppChatLoader("../data/andrea_wchat.txt")
# docs = loader.load()
# Creating the cv database for document indexing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
index_creator = VectorstoreIndexCreator(text_splitter=text_splitter)
docsearch = index_creator.from_loaders([loader])
retriever = docsearch.vectorstore.as_retriever()

Using embedded DuckDB without persistence: data will be transient


In [7]:
test_retrieval = retriever.get_relevant_documents("Trenes.")
test_retrieval

[Document(page_content='Aitor Mira on 13/11/16, 19:43: Que son los chófer del viernes\n\nAitor Mira on 13/11/16, 19:43: Lo que no sé es tamara con quien ha contactado\n\nAitor Mira on 13/11/16, 19:44: He llamado al del viernes y me ha dicho que contacte con tranvía y ellos me dicen\n\nAndrea Paricio Henares on 13/11/16, 19:44: Vale\n\nAndrea Paricio Henares on 13/11/16, 19:44: Contacto yo?\n\nAitor Mira on 13/11/16, 19:44: Ahora los llamo yo, no te preocupes\n\nAndrea Paricio Henares on 13/11/16, 19:45: No, que va\n\nAndrea Paricio Henares on 13/11/16, 19:45: Vale, pero veme diciendo\n\nAitor Mira on 13/11/16, 19:54: Ya les he llamado\n\nAitor Mira on 13/11/16, 19:54: El servicio lo presta la marina de Gandía y hasta mañana no podrán decirnos\n\nAndrea Paricio Henares on 13/11/16, 20:01: Vale\n\nAitor Mira on 13/11/16, 20:01: Mañana llamarán a administración y nos dirán\n\nAitor Mira on 13/11/16, 20:02: Les dejo un correo a los admins para que estén enterados y la recuperen', metadata=

In [8]:
wapp_andrea = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=retriever)

In [9]:
search = GoogleSearchAPIWrapper(google_api_key="", google_cse_id="")
search.run("fecha elecciones españa")

'May 29, 2023 ... La convocatoria de las comicios aparecerá publicada el martes 30 de mayo en el BOE, de forma que las elecciones generales serán el domingo 23 de\xa0... Apr 29, 2023 ... Las elecciones municipales en España se han celebrado este domingo 28 de mayo de 2023, cuatro años después de las anteriores (que fueron el 26\xa0... 7 days ago ... El próximo 23 de julio se celebran elecciones generales en España. Si no vas a estar en tu domicilio habitual en esa fecha, puedes solicitar\xa0... May 30, 2023 ... Las elecciones municipales si son a nivel general, y se celebrarán en todos los municipios de España en el mismo día. Las elecciones municipales\xa0... Apr 4, 2023 ... El próximo 28 de mayo se celebran las elecciones municipales en todo el ... del Estado el martes 4 de abril, fecha en el que entró en vigor. Jan 2, 2023 ... A día de hoy se sabe que las elecciones generales son este año, pero todavía no se sabe con exactitud la fecha en la que se celebrarán los\xa0... También se c

In [10]:
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "Whatsapp chat with Andrea",
        func=wapp_andrea.run,
        description="useful when you need information about Andrea, Aitor and his conversations. Including his tone and style."
    )
]

In [11]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [12]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [21]:
from langchain.agents import StructuredChatAgent
from langchain.chat_models import ChatOpenAI

In [22]:
chat_llm = ChatOpenAI(temperature=0)
chat_llm_chain = LLMChain(llm=chat_llm, prompt=prompt)
chat_agent = StructuredChatAgent(llm_chain=chat_llm_chain, tools=tools, verbose=True)
chat_agent_chain = AgentExecutor.from_agent_and_tools(agent=chat_agent, tools=tools, verbose=True, memory=memory)

In [20]:
input_query = """Multiplica por dos la respuesta de mi primera pregunta"""
agent_chain.run(input=input_query)



> Entering new AgentExecutor chain...
Thought: I need to calculate the answer to the first question
Action: Search
Action Input: Population of Spain
Observation: The current population of Spain is 46,807,685 as of Thursday, June 8, 2023, based on Worldometer elaboration of the latest United Nations data. · Spain 2020 ... Conversion to Islam proceeded at an increasing pace. The muladíes (Muslims of ethnic Iberian origin) are believed to have formed the majority of the population ... This is the first study to report the prevalence of general obesity and abdominal obesity (AO) in the adult population of Spain based on measurements of ... ... in the general population of Spain: results from a national survey ... and fibromyalgia in the adult Spanish population, and to assess the impact of ... Based on data from 1 January 2010, the registered population in Spain is over 46.9 million; the number of registered foreign nationals is 5.7 million, of which ... Nov 27, 2020 ... Infection fatali

'93,615,370'

In [25]:
input_query = """Multiplica por dos la respuesta de mi primera pregunta"""
response = await chat_agent_chain.arun(input=input_query)
print(response)



> Entering new AgentExecutor chain...
Thought: The original question was already answered twice, so I need to clarify which question is being referred to.
Action: Ask for clarification
Action Input: Which question are you referring to?


> Finished chain.
Thought: The original question was already answered twice, so I need to clarify which question is being referred to.
Action: Ask for clarification
Action Input: Which question are you referring to?



In [28]:
input_query = """Hola, buenas noches"""
response = await chat_agent_chain.arun(input=input_query)
print(response)



> Entering new AgentExecutor chain...
Thought: This is a greeting, not a question.
Action: None


> Finished chain.
Thought: This is a greeting, not a question.
Action: None



In [27]:
print(response)

Thought: I need to provide the answer to the original question.
Action: Search
Action Input: "current population of Spain"

